<a href="https://colab.research.google.com/github/vishruti1045/Cassava-Leaf-Disease-Detection-22IT087/blob/main/Cassava_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tez

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import os

import albumentations
import matplotlib.pyplot as plt
import pandas as pd

import tez
from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping

import torch
import torch.nn as nn
from torch.nn import functional as F

import torchvision

from sklearn import metrics, model_selection, preprocessing

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Assuming your dataset is in a CSV file named 'data.csv'
dfx = pd.read_csv('/content/drive/MyDrive/Projects/Cassava data/train.csv')

In [ ]:

# and split it into training and validation sets
df_train, df_valid = model_selection.train_test_split(
    dfx,
    test_size=0.1,
    random_state=42,
    stratify=dfx.label.values
)

# reset index on both dataframes
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

df_train.count()


,0
image_id,19257
label,19257


In [ ]:
dfx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21397 entries, 0 to 21396
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  21397 non-null  object
 1   label     21397 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 334.5+ KB


In [ ]:
dfx.label.value_counts()
#here we can see that label 3 has more bias/ samples

,count
label,
3,13158
4,2577
2,2386
1,2189
0,1087


In [ ]:
#checking for null values
dfx.isnull().sum()

,0
image_id,0
label,0


In [ ]:
df_train.shape

(19257, 2)

In [ ]:
df_valid.shape

(2140, 2)

In [ ]:
import os
from PIL import Image

# Define the directory where images are stored
image_path = "/content/drive/MyDrive/Projects/Cassava data/train_images/"

# Create a list of image paths for training
train_image_paths = [os.path.join(image_path, x) for x in df_train.image_id.values]

# Create a list of image paths for validation
valid_image_paths = [os.path.join(image_path, x) for x in df_valid.image_id.values]

# Targets for training
train_targets = df_train.label.values

# Targets for validation
valid_targets = df_valid.label.values

# Function to resize images
def resize_images(image_paths, size):
    resized_images = []
    for path in image_paths:
        img = Image.open(path)
        img = img.resize(size)
        resized_images.append(img)
    return resized_images

# Resize images before passing them to the dataset
resized_train_images = resize_images(train_image_paths, (250, 250))
resized_valid_images = resize_images(valid_image_paths, (256, 256))

# Assuming ImageDataset can accept images directly
train_dataset = ImageDataset(
    image_paths=resized_train_images,
    targets=train_targets,
    augmentations=None,
)

valid_dataset = ImageDataset(
    image_paths=resized_valid_images,
    targets=valid_targets,
    augmentations=None,
)


In [ ]:
train_dataset[0]

NameError: name 'train_dataset' is not defined

In [ ]:
# Let's see some images!

def plot_image(img_dict):
    image_tensor = img_dict["image"]
    target = img_dict["targets"]
    print(target)
    plt.figure(figsize=(10, 10))
    image = image_tensor.permute(1, 2, 0) / 255
    plt.imshow(image)

In [ ]:
plot_image(train_dataset[0])

NameError: name 'train_dataset' is not defined

In [ ]:
# augmentations library: albumentations
# Tez supports albumentations exclusively

train_aug = albumentations.Compose(
    [
        albumentations.RandomResizedCrop(256, 256),
        albumentations.Transpose(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.ShiftScaleRotate(p=0.5),
        # albumentations.Normalize(
        #    mean=[0.485, 0.456, 0.406],
        #    std=[0.229, 0.224, 0.225],
        #    max_pixel_value=255.0,
        #    p=1.0
        #)
    ]
)


# now, we set resize to None as we are doing
# resizing via augmentations
train_dataset = ImageDataset(
    image_paths=train_image_paths,
    targets=train_targets,
    resize=None,
    augmentations=train_aug,
)